In [1]:
from src.Model_Parameter import get_model_response

import re
import os
import csv
import json
from typing import List
import yaml
import openai
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm, trange
from langchain import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
import requests
import xml.etree.ElementTree as ET
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import nltk
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Download the 'punkt' resource
nltk.download('wordnet')
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
import pandas as pd
data = pd.read_parquet('falcon7b_pubmed_summarization.parquet')
articles = pd.read_parquet('articles.parquet')

In [3]:
merged_df = pd.merge(data, articles, on='record_id')
correct_summary = merged_df['mapped_ground_true'].tolist()
model_generated = merged_df['mapped_answer'].tolist()
articles = merged_df['article'].tolist()
print(len(correct_summary))
len(model_generated)

100


100

In [4]:
from rouge_score import rouge_scorer

In [ ]:
# Summarize osmosis transcripts and evaluate
import time

# Start time
start_time = time.time()

#looping to go accross the entire data for evaluation

# Initialize BLEU and ROUGE scorers
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
meteor_scores = []

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])
for i, note in enumerate(correct_summary):
    
    # Evaluate BLEU score
    bleu_score = corpus_bleu([[correct_summary[i].split()]], [model_generated[i].split()])
    bleu_scores.append(bleu_score)

    # Evaluate ROUGE scores
    reference = correct_summary[i]
    hypothesis = model_generated[i]
    rouge_scores = rouge_scorer.score(reference, hypothesis)

    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

     # Tokenize hypothesis and reference summaries
    hypothesis_tokens = word_tokenize(hypothesis)
    reference_tokens = word_tokenize(reference)

    # Evaluate METEOR score
    meteor_score_value = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score_value)

    #print("Clinical Note:")
    #print(note)
    print("\nCorrect Summary:")
    print(correct_summary[i])
    print("\nGenerated Summary:")
    print(model_generated[i])
    print("\nBLEU Score:", bleu_score)
    print("ROUGE Scores:", rouge_scores)
    print("METEOR Score:", meteor_score_value)
    print("--------------------------------------------------")

    
end_time = time.time()

In [6]:
# Calculate and print average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", avg_bleu_score)

# Calculate and print average ROUGE scores
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
print("Average ROUGE-1 Score:", avg_rouge1_score)
print("Average ROUGE-2 Score:", avg_rouge2_score)
print("Average ROUGE-L Score:", avg_rougeL_score)

# Calculate and print average METEOR score
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)
print("Average METEOR Score:", avg_meteor_score)

Average BLEU Score: 0.003649077018994482
Average ROUGE-1 Score: 0.19479337887518092
Average ROUGE-2 Score: 0.03209325966026071
Average ROUGE-L Score: 0.12295216808783835
Average METEOR Score: 0.11306053015896626


In [7]:
print(len(articles))
print(len(correct_summary))
print(len(model_generated))
print(len(meteor_scores))

100
100
100
100


In [9]:
data = {
    'articles': articles,
    'correct_summary': correct_summary,
    'model_generated': model_generated,
    'meteor_scores': meteor_scores
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Exporting the DataFrame to an Excel file
df.to_excel('Falcon_output.xlsx', index=False)